# Baseline Solution - Monte Carlo Dropout

## This notebook documents the baseline solution for ADC 2023. 

## Overview
Our challenge is to provide a conditional probability distribution for each target (7 in total) given an observation from the Ariel Space Telescope. 

Depending on the information content of the observation and the associated observation noise (which is a function of the instrument and the planetary system), the resultant error bounds on each target and their joint conditional distribution will be different.

There are many directions you can take to tackle the problem on hand. We would like to get you started with our baseline solution. Inside this notebook you will find the setup for the baseline model, ways to compute the competition score and how to package the output into the competition format.

Spectroscopic data alone are usually informative enough to provide a reasonable estiamte on the targets. After all, the trough and peaks in the spectra encoded information about the relative abundance of each gaseous species (see [Yip et al.](https://iopscience.iop.org/article/10.3847/1538-3881/ac1744>) ). The supplementary information also helps to better constrain some of the phyiscal quantities (see our discussion [here](https://www.ariel-datachallenge.space/ML/documentation/about) if you want to learn about the underlying physics :) , but I shall leave that to you. 

The baseline solution trains a CNN to output a deterministic estimate for each atmospheric target. At inference time, the network is made to produce probabilistic output by activating the dropout layers in the network (Monte Carlo Dropout, [Gal et al. 2016](https://arxiv.org/abs/1506.02142)). 

In [ ]:
!pip install taurex

After installing Taurex, restart the runtime through clicking Runtime tab at the left top, and selecting "Restart Runtime".

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # this will mount your Google Drive after your permission.

In [ ]:
import os
os.chdir('####The folder where you put all helper functions and MCDropout files####')

In [ ]:
import numpy as np # Numerical Python, for matrix operations and many other calculations
import tensorflow as tf # Google's TensorFlow library, specific for Deep learning
import pandas as pd # Creation of dataframes and facilitating data manipulation, similar to R dataframe
from tensorflow import keras # Deep learning networks and other relevant parameters
import h5py # to read and write HDF5 files
import os # operating system package, navigating through folders, retrieving files etc.
import matplotlib.pyplot as plt # main plotting library, similar to MATLAB plotting
from tqdm import tqdm # Progress monitoring bars in model creation and training/validation/test steps
from helper import *
import taurex # atmospheric retrieval framework, bayesian inverse, 
from submit_format import to_competition_format
from posterior_utils import *
from spectral_metric import *
from FM_utils_final import *
from taurex import log
taurex.log.disableLogging()
from MCDropout import MC_Convtrainer # baseline CNN network
from preprocessing import *

### Fix seed


In [ ]:
SEED=42

### Constants

In [ ]:
RJUP = 69911000
MJUP = 1.898e27
RSOL = 696340000

Retrieve TauRex folders

Retrieve HITRAN files

In [ ]:
# CIA collision induced absorption database, and cross section (xsec) files
# You should save these to specific files for CIA and XSEC, to later use them for forward modeling

from urllib.request import urlretrieve

urlretrieve('https://www.dropbox.com/sh/1njwmcqvv8zj3sy/AAA9ZSWfyFGy0y9xoVDlk2-xa/cia/HITRAN/H2-H2.db?dl=1', '####folder for CIA####/H2-H2.db') # modify the second part
urlretrieve('https://www.dropbox.com/sh/1njwmcqvv8zj3sy/AAAwLZaNnzZpRnSyyU-ctdIxa/cia/HITRAN/H2-He.db?dl=1', '####folder for CIA####/H2-He.db') # modify the second part

urlretrieve('https://www.dropbox.com/sh/1njwmcqvv8zj3sy/AADYnkfP_38GjVQ5iZP_pCXva/xsec/1H2-16O__POKAZATEL.R15000_0.3-50mu.xsec.TauREx.h5?dl=1', '####folder for XSEC####/1H2-16O__POKAZATEL.R15000_0.3-50mu.xsec.TauREx.h5') # modify the second part
urlretrieve('https://www.dropbox.com/sh/1njwmcqvv8zj3sy/AABJg05TMEUiy-EMcrJs3qdua/xsec/12C-1H4__YT34to10.R15000_0.3-50mu.xsec.TauREx.h5?dl=1', '####folder for XSEC####/12C-1H4__YT34to10.R15000_0.3-50mu.xsec.TauREx.h5') # modify the second part
urlretrieve('https://www.dropbox.com/sh/1njwmcqvv8zj3sy/AACLAVUeYkESGEVoF93lWvZya/xsec/12C-16O__Li2015.R15000_0.3-50mu.xsec.TauREx.h5?dl=1', '####folder for XSEC####/12C-16O__Li2015.R15000_0.3-50mu.xsec.TauREx.h5') # modify the second part
urlretrieve('https://www.dropbox.com/sh/1njwmcqvv8zj3sy/AABlnQoo7JE4tjABR2PKiy6Ia/xsec/12C-16O2__HITEMP.R15000_0.3-50mu.xsec.TauREx.h5?dl=1', '####folder for XSEC####/12C-16O2__HITEMP.R15000_0.3-50mu.xsec.TauREx.h5?dl=1') # modify the second part
urlretrieve('https://www.dropbox.com/sh/1njwmcqvv8zj3sy/AADCB0UTVz09Q7VcIy62KmXXa/xsec/14N-1H3__CoYuTe.R15000_0.3-50mu.xsec.TauREx.h5?dl=1', '####folder for XSEC####/14N-1H3__CoYuTe.R15000_0.3-50mu.xsec.TauREx.h5?dl=1') # modify the second part

**Retrieve training data**

In [ ]:
from urllib.request import urlretrieve

urlretrieve('https://www.ariel-datachallenge.space/static/data/FullDataset.zip', '####The folder where you put all helper functions and MCDropout files####/FullDataset.zip') # modify the second part


Extracting the content of Training & Test Data

In [ ]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/ari/FullDataset.zip', 'r') as zippy:
    zippy.extractall(path=os.getcwd())

## Read training data

In [ ]:
training_path = '####The folder where you put all helper functions and MCDropout files####/TrainingData' # modify
test_path = '####The folder where you put all helper functions and MCDropout files####/TestData' # modify
training_GT_path = os.path.join(training_path, 'Ground Truth Package')

spectral_training_data = h5py.File(os.path.join(training_path,'SpectralData.hdf5'),"r")
aux_training_data = pd.read_csv(os.path.join(training_path,'AuxillaryTable.csv'))
soft_label_data = pd.read_csv(os.path.join(training_GT_path, 'FM_Parameter_Table.csv'))

## Extract Spectral data
Spectral data lives in a h5py format, which is useful for navigating different cases, but their format makes it difficult to bulk manage them. The helper function helps to transform the h5py file into a matrix of size N x 52 x 4
where N is the number of training examples, 52 is the number of wavelength channels and 4 is the observation data

In [ ]:
spec_matrix = to_observed_matrix(spectral_training_data,aux_training_data)
print("spectral matrix shape:", spec_matrix.shape)

# Visualising a single spectrum

In [ ]:
def visualise_spectrum(spectrum):
    fig = plt.figure(figsize=(10,6))
    ## multiple by 100 to turn it into percentage. 
    plt.errorbar(x=spectrum[:,0], y= spectrum[:,1]*100, yerr=spectrum[:,2]*100 )
    ## we tend to visualise it in log-scale
    plt.xscale('log')
    plt.xlabel('Wavelength (micron)')
    plt.ylabel('Transit depth (%)')
    plt.show()

In [ ]:
visualise_spectrum(spec_matrix[1])

In [ ]:
## lets look at another one
visualise_spectrum(spec_matrix[2])

it is immediately apparent that the average transit depth between two spectra can change for over an order of magnitude. The magnitude of the uncertainty can also change accordingly ( and is a function of the planetary system, brightness of the host star and instrument response function). 

## Pre-processing

### Settings

In [ ]:
repeat = 5
threshold = 0.8 ## for train valid split.
N = 5000

We can safely discard wlgrid (wavelength grid) and wlwidth (width of wavelength) since they are unchanged in the dataset

### Extract Spectrum

In [ ]:
## extract the noise
noise = spec_matrix[:N,:,2]
## We will incorporate the noise profile into the observed spectrum by treating the noise as Gaussian noise.
spectra = spec_matrix[:N,:,1]
wl_channels = len(spec_matrix[0,:,0])
global_mean = np.mean(spectra)
global_std = np.std(spectra)

### Adding an additional feature - radius of the star 
Most of the time we know something about the planetary system before we even attempt to make an observation (we cant just point randomly with a multi-million euros instrument!). Some of these auxillary data may be useful for retrieval, here we are only using the radius of the star.

In [ ]:
## add Rstar 
Rs = aux_training_data[['star_radius_m',]]
## we would prefer to use Rsol
Rs['star_radius'] = Rs['star_radius_m']/RSOL
Rs = Rs.drop(['star_radius_m'],axis=1)
Rs = Rs.iloc[:N, :]
mean_Rs = Rs.mean()
stdev_Rs = Rs.std()

### Get targets

In [ ]:
target_labels = ['planet_radius','planet_temp','log_H2O','log_CO2','log_CO','log_CH4','log_NH3']
targets = soft_label_data.iloc[:N][target_labels]
num_targets = targets.shape[1]
targets_mean = targets.mean()
targets_std = targets.std()

## Train/valid Split

In [ ]:
ind = np.random.rand(len(spectra)) < threshold
training_spectra, training_Rs,training_targets, training_noise = spectra[ind],Rs[ind],targets[ind], noise[ind]
valid_spectra, valid_Rs, valid_targets = spectra[~ind],Rs[~ind],targets[~ind]

## Augment the dataset with noise (create multiple instances)
Observational noise from Ariel forms an important part of the challenge, any model must recognise that the observation are not absolute measurement and could vary (according to the uncertainty), as that will affect the uncertainty associated with our atmospheric targets. Here we try to incorporate these information by augmenting the data with the mean noise.

In [ ]:
aug_spectra = augment_data_with_noise(training_spectra, training_noise, repeat)
aug_Rs = np.tile(training_Rs.values,(repeat,1))
aug_targets = np.tile(training_targets.values,(repeat,1))

### Standardise the data

### spectra

In [ ]:
## standardise the input using global mean and stdev
std_aug_spectra = standardise(aug_spectra, global_mean, global_std)
std_aug_spectra = std_aug_spectra.reshape(-1, wl_channels)
std_valid_spectra = standardise(valid_spectra, global_mean, global_std)
std_valid_spectra = std_valid_spectra.reshape(-1, wl_channels)

### radius

In [ ]:
## standardise
std_aug_Rs= standardise(aug_Rs, mean_Rs.values.reshape(1,-1), stdev_Rs.values.reshape(1,-1))
std_valid_Rs= standardise(valid_Rs, mean_Rs, stdev_Rs)

### target
We are asking the model to provide estimates for 6 atmospheric targets. In this example will be performing a supervised learning task. 

In [ ]:
std_aug_targets = standardise(aug_targets, targets_mean.values.reshape(1,-1), targets_std.values.reshape(1,-1))
std_valid_targets = standardise(valid_targets, targets_mean, targets_std)

# Setup network


### hyperparameter settings


In [ ]:
batch_size= 32
lr= 1e-3
epochs = 30
filters = [32,64,64]
dropout = 0.1
# number of examples to generate in evaluation time (5000 is max for this competition)
N_samples = 5000

We followed [Yip et al.](https://iopscience.iop.org/article/10.3847/1538-3881/ac1744>) and adopted a simple CNN structure and loss function. 


In [ ]:
model = MC_Convtrainer(wl_channels,num_targets,dropout,filters)

### Compile model and Train!

In [ ]:
## compile model and run
model.compile(
    optimizer=keras.optimizers.Adam(lr),
    loss='mse',)
model.fit([std_aug_spectra,std_aug_Rs], 
          std_aug_targets, 
          validation_data=([std_valid_spectra, std_valid_Rs],std_valid_targets),
          batch_size=batch_size, 
          epochs=epochs, 
          shuffle=False,)

Optional saving for model to a file, or loading a model from a file

In [ ]:
# Saving a model for later use
model.save("model.h5") # you can save a network to a file for being able to use it later
model = keras.models.load_model("model.h5") # This will load the previously saved model

Evalute model with validation data

In [ ]:
## select the corresponding GT for the validation data, and in the correct order.
index= np.arange(len(ind))
valid_index = index[~ind]

In [ ]:
instances = N_samples
y_valid_distribution = np.zeros((instances, len(std_valid_spectra), num_targets ))
for i in tqdm(range(instances)):
    
    y_pred_valid = model([std_valid_spectra,std_valid_Rs],training=True)
    y_valid_distribution[i] += y_pred_valid

In [ ]:
y_valid_distribution = y_valid_distribution.reshape(-1,num_targets)

In [ ]:
y_pred_valid_org = transform_and_reshape(y_valid_distribution,targets_mean, targets_std,instances,N_testdata=len(std_valid_spectra))

In [ ]:
tr1 = y_pred_valid_org
# weight takes into account the importance of each point in the tracedata. for now we just assume them to be equally weighted
weights1 = np.ones((tr1.shape[0],tr1.shape[1]))/np.sum(np.ones(tr1.shape[1]) )

In [ ]:
trace_GT = h5py.File(os.path.join(training_GT_path, 'Tracedata.hdf5'),"r")

## posterior scores 
This score accounts for 80% of the final score and it is based on the entire dataset.

In [ ]:
posterior_scores = []
bounds_matrix = default_prior_bounds()
for idx, pl_idx in enumerate(valid_index):
    tr_GT = trace_GT[f'Planet_train{pl_idx+1}']['tracedata'][()]
    weights_GT = trace_GT[f'Planet_train{pl_idx+1}']['weights'][()]
    ## there are cases without ground truth, we will skip over them for this baseline
    ## but every example in leaderboard and final evaluation set will have a complementary ground truth
    if np.isnan(tr_GT).sum() == 1:
        continue
    # compute posterior loss
    score = compute_posterior_loss(tr1[idx], weights1[idx], tr_GT, weights_GT, bounds_matrix)
    posterior_scores.append(score)
avg_posterior_score = np.mean(posterior_scores)    

In [ ]:
print(avg_posterior_score)

## spectral scores 
This score accounts for 20% of the final score and it is based on a pre-selected, classified subset of the entire dataset.
It takes a while to compute the score, even for 100 samples, so we will randomly draw 20 in this case to illustrate the idea.

**CAUTION: To use this metric you must have taurex and their linelists available on your local environment. Please refer to README.MD on the github repo for more information**

In [ ]:
N_examples = 20 ## number of test examples to go through
N_samples = 10 ## number of quantiles to sample (fixed to 10 in the competition)
q_list = np.linspace(0.01,0.99,N_samples)
## beta - weight of the posterior loss [0,1], and the weight of spectral loss will decrease accordingly. 
beta = 0.8

In [ ]:
## Path variables
opacity_path="####folder for XSEC####"
CIA_path="####folder for CIA####"

In [ ]:
## read in spectral grid
ariel_wlgrid, ariel_wlwidth, ariel_wngrid, ariel_wnwidth = ariel_resolution()
## Initialise base T3 model for ADC2023
fm = initialise_forward_model(opacity_path, CIA_path)

In [ ]:
# raed auxillary information from the input file (Provided from ADC2023)
aux_df = aux_training_data
# ensure the dimensionality matches forward model's input.
Rs = aux_df['star_radius_m']/RSOL
# Rp = aux_df['planet_radius_m']/RJUP
Mp = aux_df['planet_mass_kg']/MJUP

In [ ]:
## select few random validation data for spectral loss computation

In [ ]:
spectral_scores = []
bounds_matrix = default_prior_bounds()
for idx, pl_idx in enumerate(valid_index):
    ## put an early stop here as it will take forever to go through 5000 examples. 
    if idx == 20:
        break
    tr_GT = trace_GT[f'Planet_train{pl_idx+1}']['tracedata'][()]
    weights_GT = trace_GT[f'Planet_train{pl_idx+1}']['weights'][()]
    # again to avoid unlabelled data
    if np.isnan(tr_GT).sum() == 1:
        continue

    proxy_compute_spectrum = setup_dedicated_fm(fm, idx, Rs, Mp, ariel_wngrid, ariel_wnwidth )

    score = compute_spectral_loss(tr1[idx], weights1[idx], tr_GT,weights_GT,bounds_matrix,proxy_compute_spectrum,q_list)
    spectral_scores.append(score)
avg_spectral_score = np.mean(spectral_scores)

In [ ]:
final_score = (1-beta)*avg_spectral_score + beta *avg_posterior_score
print(f"final loss is {final_score:.4f}")

# Generate prediction for leaderboard

### load leaderboard data

In [ ]:
spec_test_data = h5py.File(os.path.join(test_path,'SpectralData.hdf5'),"r")
aux_test_data = pd.read_csv(os.path.join(test_path,'AuxillaryTable.csv'))

### same pre-processing as before...

In [ ]:
test_spec_matrix = to_observed_matrix(spec_test_data,aux_test_data )

In [ ]:
std_test_spectra = standardise(test_spec_matrix[:,:,1], global_mean, global_std)

In [ ]:
test_Rs = aux_test_data[['star_radius_m']]
## we would prefer to use RSol 
test_Rs['star_radius'] = test_Rs['star_radius_m']/RSOL
test_Rs = test_Rs.drop(['star_radius_m'],axis=1)
std_test_Rs= standardise(test_Rs, mean_Rs, stdev_Rs)

## Predict and postprocess
We will sample 5000 times by activating dropout at inference phase. This is done explicitly via training = True. Note that in the competition, any sample size bigger than 5000 will NOT be accepted. However, the sample size must have a minimum of 1000 points to be a valid submission. 

In [ ]:
tf.keras.utils.set_random_seed(SEED)

In [ ]:
instances = N_samples+ 4990 # for making 5000 instances, the challenge requires instances between 1000 and 5000
y_pred_distribution = np.zeros((instances, len(std_test_spectra), num_targets ))
for i in tqdm(range(instances)):
    
    y_pred = model([std_test_spectra,std_test_Rs],training=True)
    y_pred_distribution[i] += y_pred

In [ ]:
y_pred_distribution = y_pred_distribution.reshape(-1,num_targets)

In [ ]:
#y_pred_test_org = transform_and_reshape(y_pred_distribution,targets_mean, targets_std,instances,N_testdata=len(std_test_spectra))
y_pred_test_org = transform_and_reshape(y_pred_distribution,targets_mean, targets_std,instances,N_testdata=len(std_test_spectra))

## Package output into desired format
We follow specific formats in the competition, to help make the process as painless as possible, we have included a few helper functions to make sure you have the right format in place for the submission. 

In [ ]:
tracedata = y_pred_test_org
# weight takes into account the importance of each point in the tracedata. 
weight = np.ones((tracedata.shape[0],tracedata.shape[1]))/np.sum(np.ones(tracedata.shape[1]) )

In [ ]:
submission = to_competition_format(tracedata, 
                                        weight, 
                                        name="submission.hdf5") # you may modify this for different submissions

## check!

## Future work

There are different direction to take from here on, let us summarise the shortcomings of this model:
- The data preprocessing is quite simplistic and could have invested with more efforts.
- we have only used 5000 data points, instead of the full dataset
- we didnt train the model with results from the retrieval ( Tracedata.hdf5), which are the GT for this competition.
- The conditional distribution from MCDropout is very restricted and Gaussian-like
- So far we havent considered the atmospheric targets as a joint distribution
- We have only used stellar radius from the auxillary information
- We have not done any hyperparameter tuning 
- the train test split here is not clean, as in, we split the data after we have augmented the data, which results in information leakage to the validation data. There is no leakage to the test data though.